In [1]:
"""
By default the below script will download most recent 100 comments of a youtube video
You can change the default filter (line 30 onwards)

Variables :
COMMENT_LIMIT : How many comments you want to download 
SORT_BY_POPULAR : filter comments by popularity (0 for True , 1 for false)
SORT_BY_RECENT : filter comments by recently posted (0 for True , 1 for false)
"""

import pandas as pd                                                      #for working with data frames
import json
import os                                                                ##for file and system operations
import sys                                                               ##for file and system operations
import re
import time                                                             #for time-related operations

import csv
import datetime

import requests                                                             #for making HTTP requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# csv file name
FILE_NAME = 'ENGLISH_youtube_comments.csv'

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent
SORT_BY_RECENT = 0
# set comment limit
COMMENT_LIMIT = 100


###
### The script defines regular expressions (YT_CFG_RE and YT_INITIAL_DATA_RE)
### for extracting YouTube configuration data and initial data from the HTML response.
##
YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

In [2]:
def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default
###
# The regex_search function takes three parameters: text, pattern, and group. 
# It uses regular expressions to search for a pattern in the given text and returns the matched group. 
# If no match is found, it returns the default value.
###




def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)
# The ajax_request function takes several parameters: session, endpoint, ytcfg, retries, and sleep. 
# It constructs the URL for the AJAX request using the endpoint information and the base URL of YouTube.
# It prepares the data payload for the request using the ytcfg information and the continuation token from the endpoint.

# The function then makes a POST request to the constructed URL with the necessary parameters.
# If the response status code is 200, it returns the JSON response. 
# If the response status code is 403 or 413, it returns an empty dictionary.
# Otherwise, it waits for a specified amount of time (defined by sleep) and retries the request a specified number of times (defined by retries).


            
            
            
import requests         # requests for making HTTP requests
from bs4 import BeautifulSoup    # for parsing HTML
import json      # for working with JSON data
import re        # for regular expressions
import time      # for introducing delays        
            

## The function below to download comments

            
def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT     # starts by creating a session object and setting the User-Agent header
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)
## It makes a GET request to the YouTube video URL and checks if the request is redirected.
## If it is redirected, it sets a cookie and makes another request to the URL.
        
    html = response.text  # The HTML response from the request is stored in the html variable.
    

    #USE BeautifulSoup to parse the HTML to extract the title of video
    soup = BeautifulSoup(html,'html.parser')
    video_title = soup.title.string.strip()    
        
    
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:            # Unable to extract configuration
        return 
    if language:        # updates the language in the ytcfg data.
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language
    ##  extracts the YouTube configuration data (ytcfg) using regular expressions and JSON parsing.
    ##  If the configuration data cannot be extracted, the function returns.
        
    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))
    
    #The script extracts the initial data of the video using regular expressions and JSON parsing.
    

    section = next(search_dict(data, 'itemSectionRenderer'), None) 
    #searches for the section containing the comments and retrieves the first continuation item renderer.
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:        
        return             #If there is no continuation item renderer, it indicates that comments are disabled, and the function returns.

    needs_sorting = sort_by != SORT_BY_POPULAR
    # If sorting is required (not sorting by popularity), it searches for the sort menu and selects the desired sorting option. 
    # If the sorting fails, an exception is raised.
    
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])
        ###
        # It retrieves the comments and continuation items using the ajax_request function. 
        # It processes the response by iterating over the comments and extracting relevant information 
        # such as the comment ID, text, timestamp, author, channel, votes, photo, and heart status.
        ###
        for comment in reversed(list(search_dict(response, 'commentRenderer'))):  #The reversed function is then used to reverse the order of the list, so the comments are processed in reverse order (from the most recent to the oldest).
               
        ##This line iterates over the list of comment renderers extracted from the YouTube API response.
        ##The search_dict function is used to find all instances of the "commentRenderer" key in the response JSON, and the result is converted to a list.
        

       
             yield {'Video_title':video_title,   ##The yield statement is used to define a generator function. When this statement is encountered, it pauses the function's execution, "yields" a value, and remembers its current state.
                   
                   'Comment_id': comment['commentId'],
                   
                   'Comment_content': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]), 
                    #The .get() method is used to safely access nested properties of the comment object to handle cases where a property might be missing.
                    'Time': comment['publishedTimeText']['runs'][0]['text'],
                   'Username': comment.get('authorText', {}).get('simpleText', ''),
                   'Channel_user': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'Likes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'Photo_thumbnails': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'Hearted_by_video_publisher': next(search_dict(comment, 'isHearted'), False)}
        
        ###
        # The next(search_dict(comment, 'isHearted'), False) expression is used to check if the comment has been hearted (liked) by the viewer.
        # The search_dict function is used to find the "isHearted" key in the comment object, and the next function returns its value. 
        # If the key is not found, the default value is set to False.
        ###
   
       ##The extracted information is yielded as a dictionary for each comment.
    
    # The dictionary is created using a dictionary literal syntax (curly braces {}), and the values are extracted from the comment object, 
    # which represents an individual comment renderer in the loop.
        
        time.sleep(sleep)   #After each iteration, the function introduces a delay specified by the sleep parameter.

        
        
def search_dict(partial, search_key): # partial:the nested dictionary or list to search, search_key: the key to search for
    stack = [partial]    #The stack is used to keep track of the nested levels of the dictionary or list that need to be explored.
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)
###
#  If current_item is a dictionary, the function iterates over its key-value pairs. 
#  For each key-value pair, it checks if the key matches the search_key. If there is a match, it yields the corresponding value.
#  If the key doesn't match, it pushes the value onto the stack to explore it further.
###

###
#  If current_item is a list, the function iterates over its elements.
#  For each element, it pushes the value onto the stack to explore it further.
###

###
#  This function yields the matching values one at a time as they are found, 
#  rather than finding all matches at once and returning them as a list.
#  This can be useful when working with large nested data structures, as
#   it avoids the need to load the entire structure into memory at once.
###

In [3]:
import pandas as pd
import os
import sys
import time
import json
from IPython.display import display    #used for displaying the comments DataFrame in a Jupyter Notebook or IPython environment.

# Constants
COMMENT_LIMIT = None  # Set the limit for the number of comments to download
FILE_NAME = 'ENGLISH_youtube_comments.csv'  # Specify the file name for the CSV file

def main(url):
    """
    This function will save the comments into a DataFrame and output a CSV file
    By default, it will append the comments in the CSV file, not overwrite them.
    To preview the comments in JSON, uncomment the lines - 22 and 23.
    """
    df_comment = pd.DataFrame()
    #  a new DataFrame df_comment is created to store the comments.
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading YouTube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = pd.concat([df_comment, pd.DataFrame([comment])], ignore_index=True)
            # comments overview in JSON
            # comment_json = json.dumps(comment, ensure_ascii=False)
            # print(comment_json)

            count += 1

            if limit and count >= limit:
                break

        print("DataFrame Shape:", df_comment.shape)
        display(df_comment)

        if not os.path.isfile(FILE_NAME):
            df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
        else:  # else it exists, append without writing the header
            df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

In [22]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
# dumping youtube comments

""" 
Dump comments to a csv  from a single video

"""
youtube_URL = 'https://www.youtube.com/watch?v=5kSc3eWdZdc'
main(youtube_URL)



import csv

# the function add the encoding parameter "utf-8-sig"
# The "utf-8-sig" encoding includes a UTF-8


# 'w' will create a new csv il
def save_comments_to_csv(comments, filename):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as csvfile:
        fieldnames = ['Video_title','Comment_id', 'Comment_content', 'Time',
                      'Username', 'Channel_user', 'Likes', 'Photo_thumbnails', 'Hearted_by_video_publisher']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for comment in comments:
            #####
            
            
            
            #####
            writer.writerow(comment)

# Call the download_comments function and save the results to a CSV file
YOUTUBE_VIDEO_URL = 'https://www.youtube.com/watch?v=5kSc3eWdZdc'
comments = download_comments(YOUTUBE_VIDEO_URL)

# Specify the filename for the CSV file
filename = 'ENGLISH_youtube_comments.csv'

# Save the comments to the CSV file
save_comments_to_csv(comments, filename)





DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,How to Build Muscle in HALF The Time (Using Sc...,UgwT7F8XkOIRp2E5RNV4AaABAg,Here's a link to the time-saving workouts you ...,4 日前,@JeremyEthier,UCERm5yFZ1SptUEU4wZ2vJvw,25,https://yt3.ggpht.com/ytc/APkrFKbBodUzGTci1r8S...,False
1,How to Build Muscle in HALF The Time (Using Sc...,UgzckqYrzG-mrr6uoRp4AaABAg,First comment from India 🇮🇳 Indians are gifted...,4 日前 (已編輯),@shardoolkaith9688,UCclhxi1WGOwY4GuA-PSgUoA,102,https://yt3.ggpht.com/weEUErJzoaQt3gzXgswj_zbS...,False
2,How to Build Muscle in HALF The Time (Using Sc...,Ugw-Feb4--wmV5HjU8d4AaABAg,00:20 Strategy 1 Minimal volume\n04:30 Strateg...,4 日前 (已編輯),@juliafloridausa,UCwtdHRcJtkOb9A3ZCrASo0g,82,https://yt3.ggpht.com/ytc/APkrFKYiYkMWvK00GM6X...,False
3,How to Build Muscle in HALF The Time (Using Sc...,UgwWCObkaSIFOVZVGk14AaABAg,Hey Jeremy! With all the new studys and scienc...,4 日前,@FeralVyrux,UC6cBXzN8JZGZpxklH14VBqA,27,https://yt3.ggpht.com/ytc/APkrFKZ66HBjOf6hifk-...,False
4,How to Build Muscle in HALF The Time (Using Sc...,Ugx5_EwY8B9Vg5DDr854AaABAg,Some of my best progress was in a home gym doi...,4 日前,@jamaallewis3520,UCG3htk6YFDsliBck0PE6gMA,3,https://yt3.ggpht.com/ytc/APkrFKacTctnc9p6doY8...,False
5,How to Build Muscle in HALF The Time (Using Sc...,UgzNhcTEKRe-Z0aWuZ54AaABAg,This is really good to know and matches with w...,3 日前,@Erydanus,UCpKX47qNIVXCyJj4IOENr6w,3,https://yt3.ggpht.com/IfiRMFgh9VjE_D1cm7QHjrRb...,False
6,How to Build Muscle in HALF The Time (Using Sc...,UgxuwB1g3xJFFvNGshl4AaABAg,Thank you so much for the free workout program...,2 日前,@monicagarner8313,UCb5rU9h-YsF6GBAdURejuAg,1,https://yt3.ggpht.com/ytc/APkrFKYNtCNUpZ5dzKBS...,False
7,How to Build Muscle in HALF The Time (Using Sc...,UgwHWFhlfMvDNpnAGoB4AaABAg,"Hi Jeremy, first of all I want to thank you fo...",4 日前,@Valentinazs,UCKZ6XjXPAUo-I0SSBYc9lWA,0,https://yt3.ggpht.com/c1X8SkUuVd1N-mokVh37CQbN...,False
8,How to Build Muscle in HALF The Time (Using Sc...,UgzCCpEhzA14tAwiJf54AaABAg,For the last couple years I've been doing a 3-...,3 日前,@DThunder85,UC53q4TocqWXRPYSd11P_l8Q,6,https://yt3.ggpht.com/ytc/APkrFKbhhQwwQn_6KQUF...,False
9,How to Build Muscle in HALF The Time (Using Sc...,Ugxb80qZPgVDxITSAst4AaABAg,What a very well done video. Congrats!,1 日前,@accido002,UCbc6kEY4TZfyfAtDO4kl21A,0,https://yt3.ggpht.com/ytc/APkrFKZjjdnM4HQ17ZGf...,False



[0.99 seconds] Done!


In [5]:
df_comment = pd.read_csv('ENGLISH_youtube_comments.csv')
df_comment.shape

(20, 9)

In [6]:
"""
Dump to a csv from a a list with video links
"""
ytb_video_list = ['https://www.youtube.com/watch?v=yWnacRo2VbA',
                  'https://www.youtube.com/watch?v=bcuoMCCY1qw',
                  'https://www.youtube.com/watch?v=MIQmrBxp2-E',
                  'https://www.youtube.com/watch?v=FB5EdxAGxQg',
                  'https://www.youtube.com/watch?v=F-v1IXvFd_4',
                  'https://www.youtube.com/watch?v=fis26HvvDII',
                  'https://www.youtube.com/watch?v=FjrKMcnKahY',
                  'https://www.youtube.com/watch?v=RBSUwFGa6Fk',
                  'https://www.youtube.com/watch?v=sOJRJtM_iu0',
                  'https://www.youtube.com/watch?v=PXub4lr-9J8'
                  ]

for video_link in ytb_video_list:
    main(video_link)



    
import csv


"""
To avoid overwriting the existing comments in the CSV file and append new comments,
can modify the save_comments_to_csv function to open the file in append mode ('a') instead of write mode ('w').
"""


def save_comments_to_csv(comments, filename):
    with open(filename, 'a', encoding='utf-8-sig', newline='') as csvfile:
        fieldnames = ['Video_title','Comment_id', 'Comment_content', 'Time',
                      'Username', 'Channel_user', 'Likes', 'Photo_thumbnails', 'Hearted_by_video_publisher']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for comment in comments:
            writer.writerow(comment)

# List of YouTube video URLs
YOUTUBE_VIDEO_URLS = [
    'https://www.youtube.com/watch?v=yWnacRo2VbA',
    'https://www.youtube.com/watch?v=bcuoMCCY1qw',
    'https://www.youtube.com/watch?v=MIQmrBxp2-E',
    'https://www.youtube.com/watch?v=FB5EdxAGxQg',
    'https://www.youtube.com/watch?v=F-v1IXvFd_4',
    'https://www.youtube.com/watch?v=fis26HvvDII',
    'https://www.youtube.com/watch?v=FjrKMcnKahY',
    'https://www.youtube.com/watch?v=RBSUwFGa6Fk',
    'https://www.youtube.com/watch?v=sOJRJtM_iu0',
    'https://www.youtube.com/watch?v=PXub4lr-9J8'
    # Add more video URLs as needed
]

# Specify the filename for the CSV file
filename = 'ENGLISH_youtube_comments.csv'

# Initialize an empty list to store all comments
all_comments = []

# Process each video URL and combine the comments
for video_url in YOUTUBE_VIDEO_URLS:
    comments = download_comments(video_url)
    all_comments.extend(comments)

# Save the combined comments to the CSV file
save_comments_to_csv(all_comments, filename)

DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,The BEST Way to Use Cardio to Lose Fat (Based ...,UgwMyzxX5GPtK1fKtvp4AaABAg,Hope you guys enjoyed this one! See below for ...,1 個月前,@JeremyEthier,UCERm5yFZ1SptUEU4wZ2vJvw,108,https://yt3.ggpht.com/ytc/APkrFKbBodUzGTci1r8S...,False
1,The BEST Way to Use Cardio to Lose Fat (Based ...,UgzOppGVYCPDxygRmBt4AaABAg,I love the fact that you backed everything wit...,1 個月前,@CrazyQInfidel,UCSn3XPA_PoSd4sWxO2mZq4Q,792,https://yt3.ggpht.com/_cpXWTPu781B14ZF3hZJ5Ayi...,False
2,The BEST Way to Use Cardio to Lose Fat (Based ...,UgwtL4fvoalYe6o9G9N4AaABAg,By far the biggest factor that’s always overlo...,1 個月前,@Grumpyoldman666,UC1cksdTTpBk9wm4XtsrkSgg,82,https://yt3.ggpht.com/li5QBjUju7JXjenDqRuNptqv...,False
3,The BEST Way to Use Cardio to Lose Fat (Based ...,Ugy-vfUdqpdgrxgc2KZ4AaABAg,I dropped 100 lbs running 3 miles a day and li...,1 個月前,@mamajupiter6158,UCPNtre80nt_CwiuiBjvFKaw,35,https://yt3.ggpht.com/4EVytBgyi5vwdUvJubqkm89B...,False
4,The BEST Way to Use Cardio to Lose Fat (Based ...,UgxKbh9tebDft4KpjNt4AaABAg,Extremely useful. I hit plateau a few times an...,2 星期前,@rheagalsim7497,UCeDuxeQq0aZ92rOLKb29wXw,3,https://yt3.ggpht.com/ytc/APkrFKYXolxfJkOTbSKw...,False
5,The BEST Way to Use Cardio to Lose Fat (Based ...,UgzSdnX2EcgHsQvKCf94AaABAg,I really appreciate the amount of research and...,1 個月前,@tifi29,UCTJalDfmHC8e_gwvvTtvkLg,13,https://yt3.ggpht.com/ytc/APkrFKZ6CtCUBmI2_Sjz...,False
6,The BEST Way to Use Cardio to Lose Fat (Based ...,UgzwD3ub722ZZ8_yo414AaABAg,I will always do a bit of cardio at the end of...,1 個月前,@Nathan.007,UCKzmgaz7rGUa0qKaCLtj54A,66,https://yt3.ggpht.com/ytc/APkrFKZWG3ofMZFsn8pA...,False
7,The BEST Way to Use Cardio to Lose Fat (Based ...,UgxYwWURl85MXNGM_3Z4AaABAg,I strongly agree with this. Based on my experi...,2 星期前,@derekfume8810,UCFab3LNzbCZDmGIZjW2Nh3w,2,https://yt3.ggpht.com/ytc/APkrFKYcnRVxdbs-jfQn...,False
8,The BEST Way to Use Cardio to Lose Fat (Based ...,UgyhnLTbvGNpvAsh5Pd4AaABAg,I’m a man of Science but I wanna say this from...,1 個月前,@cosmicbeing348,UCo2qnpEFPqWIXffeI41TdGw,107,https://yt3.ggpht.com/zIAxRgqLDKyWn5_UZuSB1wMz...,False
9,The BEST Way to Use Cardio to Lose Fat (Based ...,Ugy3sfu2peJKzwWnQPB4AaABAg,Jeremy…thank you for all the research you do t...,2 星期前,@annmartinez3174,UCDS-ZzDdco788jE_tiv7DNQ,0,https://yt3.ggpht.com/ytc/APkrFKbQKW2jEV6GVDIY...,False



[0.95 seconds] Done!
DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,How Long It ACTUALLY Takes to Burn Off A Big M...,UgxxqR4Bew6GL88wKXh4AaABAg,Hope you enjoyed this one! Comment below what ...,1 個月前,@JeremyEthier,UCERm5yFZ1SptUEU4wZ2vJvw,139,https://yt3.ggpht.com/ytc/APkrFKbBodUzGTci1r8S...,False
1,How Long It ACTUALLY Takes to Burn Off A Big M...,UgxVU7jGKA-ryvJA-2l4AaABAg,"Dude, I would love to see you turn this into a...",1 個月前,@Noctarilfate,UCXAxAF6q6CFfdJ7Arrv_VbQ,663,https://yt3.ggpht.com/ytc/APkrFKaMuKJOTVMgiXsS...,False
2,How Long It ACTUALLY Takes to Burn Off A Big M...,UgzZQ9gHGiX3vlYuJpl4AaABAg,This was a really fun video. As a long time su...,1 個月前,@lincolnbeckett8791,UCtS8uUwus_c7ydVwxa9QgKg,701,https://yt3.ggpht.com/ytc/APkrFKaOagbB25kQIEDT...,False
3,How Long It ACTUALLY Takes to Burn Off A Big M...,UgxchTwF__w6fyV35B14AaABAg,cam's philosophy towards exercise is great. he...,1 個月前,@fidelsantiago98,UCYhK3AnlmQ5UPLGP7NPcuQA,250,https://yt3.ggpht.com/ytc/APkrFKbFboS4mOjNAxNV...,False
4,How Long It ACTUALLY Takes to Burn Off A Big M...,UgwXtmBnMxuGRfCZ-794AaABAg,This video is an eye-opener about how importan...,1 個月前,@mojo1053,UCc9LLD0ZHv_d_IfeNd1fLSw,178,https://yt3.ggpht.com/ytc/APkrFKYgO5gprqsXqanu...,False
5,How Long It ACTUALLY Takes to Burn Off A Big M...,Ugxq0NATy-dD8sQUJoJ4AaABAg,So good dude! Stoked to have been a part of th...,1 個月前 (已編輯),@GoalGuys,UC6MXE0Px3m1aI4vI0pLWzQg,270,https://yt3.ggpht.com/ed0nBG_bw7LBv3jMvfwc-_5p...,True
6,How Long It ACTUALLY Takes to Burn Off A Big M...,UgzKLVSbBD22Zqv5eTx4AaABAg,Jeremy burned 14.6 calories a minute. \nNicole...,1 個月前,@shivaraj108,UChtG7M_hCTwm-nCqn_-uSQw,272,https://yt3.ggpht.com/ytc/APkrFKYo_5FKlTT2iCmv...,False
7,How Long It ACTUALLY Takes to Burn Off A Big M...,UgznxK1wkjK5r08anXx4AaABAg,I love that you have a sense of humor as well ...,1 個月前,@SolomonClarkeFitness80,UCzr_Uux3V66OUAL_ynwmiiA,140,https://yt3.ggpht.com/pdam5TTlTEx9hI5rkU9dWkiT...,False
8,How Long It ACTUALLY Takes to Burn Off A Big M...,UgySDggzDyaUE5mU5h14AaABAg,"Very visually helpful video, showing exactly h...",1 個月前,@BethVonBlack,UC8-p18VTPms6YOebsAMAi2g,58,https://yt3.ggpht.com/4nXY6Dk6rJBT_9OWd2Oye9Tw...,False
9,How Long It ACTUALLY Takes to Burn Off A Big M...,UgzHdeydvdmbJ0A_3pd4AaABAg,I love how Nicole and Jeremy are doing cardio ...,1 個月前,@hexagonshorts2186,UCnbzDIIj-cVTVKE_izk5a_w,24,https://yt3.ggpht.com/cINN9UrTeQy8-lCSGPvByEq_...,False



[1.03 seconds] Done!
DataFrame Shape: (12, 9)


...

,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,Simple Login App in Android Studio | 2023 - Yo...,UgyZhAkXYMNdyrYSLYt4AaABAg,Check this next video to make Sign In with goo...,1 年前,@EasyTuto1,UCWwkUVHo7ryXLWGuwXY_uDA,24,https://yt3.ggpht.com/ytc/APkrFKabrpkaChTvLt2o...,False
1,Simple Login App in Android Studio | 2023 - Yo...,UgyzFgj47h8IqER-bxl4AaABAg,very clear explanation. You somehow managed to...,8 個月前,@zainabazmi3865,UCAijTZ3z-OL9ulejbdi1khA,8,https://yt3.ggpht.com/ytc/APkrFKakk33SLlL6J-B6...,True
2,Simple Login App in Android Studio | 2023 - Yo...,Ugyvpar7R3QhMTqQFTJ4AaABAg,watched this for the first time and I feel lik...,10 個月前,@revathikajuluri1259,UC-_pNYJ-giaXv-IBslm699w,13,https://yt3.ggpht.com/ytc/APkrFKZVAyjQFmQAkxqq...,False
3,Simple Login App in Android Studio | 2023 - Yo...,UgzooRZwsIvw-BTsh3N4AaABAg,Quick and well-explained tutorial. Very Nice!,1 年前,@The_Snailman,UCx2fPYCXZmBYTqKj2DzTuAQ,2,https://yt3.ggpht.com/ytc/APkrFKayTbpFzFwj3zVC...,True
4,Simple Login App in Android Studio | 2023 - Yo...,UgzAIBX9QH4E9_5GoIh4AaABAg,"Thank you. Very direct, clear and concise.",1 年前,@EricBrockway,UCGiKgQeAAauwV8OSTO9eMUw,15,https://yt3.ggpht.com/ytc/APkrFKZbSzK4Yy-SwLug...,True
5,Simple Login App in Android Studio | 2023 - Yo...,UgzEVcUuBj9qI-maa3t4AaABAg,"Beautiful tutorial just wow...simple, easy and...",7 個月前,@dutchboe385,UCzXa7qAl6iFerqZvgYcG-TA,1,https://yt3.ggpht.com/RzyFpcK9Mvrns_Gi_nP_1fiu...,True
6,Simple Login App in Android Studio | 2023 - Yo...,Ugy1hIKnguYHnBAcy0h4AaABAg,It’s took 11 hours but I done it 😂\n\nGreat vi...,2 個月前,@user-kw4zh7ms3z,UCPVMtqz4oSrHqH_w2wrqdmQ,0,https://yt3.ggpht.com/ytc/APkrFKbtmJegy-oiClKy...,False
7,Simple Login App in Android Studio | 2023 - Yo...,UgwxzQ3aorDIM8zhO1F4AaABAg,Clear explanation🔥. \nGood job bro. \nthank yo...,5 個月前,@denuwanthushmika5493,UCvLdv98YQnHtORKgye8wOlA,0,https://yt3.ggpht.com/dqkwW7Pw9hYHYsOLWb8t4gKn...,False
8,Simple Login App in Android Studio | 2023 - Yo...,UgygYv3dv_s2uKPyOBx4AaABAg,It's easy to follow your steps. May I know whe...,5 個月前,@nhihuynh3869,UC7txjq8zfb3oHoWqq12bRZw,0,https://yt3.ggpht.com/ytc/APkrFKZ9JmMHqVK22-1v...,False
9,Simple Login App in Android Studio | 2023 - Yo...,UgzJCLvepOs0EblQXa94AaABAg,excelente aporte tienes alguno para registro?,1 年前,@davidshum1938,UCy-NzYBOH4HEAOpWMjiLTQA,3,https://yt3.ggpht.com/ytc/APkrFKbSkM8cj_ihNJ_z...,False



[0.99 seconds] Done!
DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgwZ0J2J7nsx2s3c62x4AaABAg,Here's a link to the downloadable meal plan: h...,2 個月前,@JeremyEthier,UCERm5yFZ1SptUEU4wZ2vJvw,108,https://yt3.ggpht.com/ytc/APkrFKbBodUzGTci1r8S...,False
1,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgzJR2QjrgyyvyEEtsF4AaABAg,Would love to see a cheap bulking meal plan as...,2 個月前 (已編輯),@simon_liu,UCdhlqNS9CdcEoXj1VAqWz7w,792,https://yt3.ggpht.com/5ESQX3QkwBGLJxB-7vCgDx_r...,True
2,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgyF9teysCf6ImhvKpN4AaABAg,"Jeremy, you are the man! The fact that you rel...",2 個月前,@PvPaddict007,UC33P_4fvl8_rSJBaQfxS0Cw,374,https://yt3.ggpht.com/ytc/APkrFKZ_8-cBErMjIWAw...,True
3,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgwiQzl1ELLQqpQiO_J4AaABAg,All 4 recipe time stamps \n\nMeal 1 Overnight ...,2 個月前,@Masked2438,UCl3dkEbfNQgROJrgCnY0YOA,108,https://yt3.ggpht.com/fEzDVVtMo2aueVdUzKWeGZws...,False
4,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,Ugz-6QvlFlbzTuYDb6N4AaABAg,Really applaud the effort you go through to pu...,2 個月前,@juicebox86,UCjOXVb_jCG-r1mzlUyuBgbw,37,https://yt3.ggpht.com/ytc/APkrFKYzBR91qlAhKau3...,False
5,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,Ugx2zOZH9vd8dQ6JwUR4AaABAg,I found you as I am educating myself to exerc...,2 個月前 (已編輯),@calamityjehn,UCVttyfpF_gCrK7HRo5smvmw,32,https://yt3.ggpht.com/ytc/APkrFKaes6QBWLbo1WVX...,False
6,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgxXY92Xg3uTlljmibB4AaABAg,Amazing Jeremy! Great video with easy-to-follo...,2 個月前,@MECHW0LF,UCVrJ3k_3mkBzDoT-MOuHpIQ,4,https://yt3.ggpht.com/ytc/APkrFKYsmSOWeLJjR-Fd...,False
7,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgwsRAHW2-4bidWyiNR4AaABAg,Great video! So many incredibly good basics to...,3 星期前 (已編輯),@mr.a.,UCvfLLfV4Viggvio0hSCk2sg,0,https://yt3.ggpht.com/ytc/APkrFKYLw43YbjqNQSRW...,False
8,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,Ugx6wgyYh6FFz9mtfmt4AaABAg,This might be the best video you ever made. Th...,2 個月前,@lifeisgood070,UC-5CUMxi1GvxZ_LDjAGa0_Q,4,https://yt3.ggpht.com/ytc/APkrFKZaGhBmgG2ARQK8...,False
9,The CHEAPEST Meal Plan to Lose Fat (HEALTHY & ...,UgyfHhHb1aG0_8OG96t4AaABAg,You are literally the best. I appreciate you a...,2 個月前,@dgoli,UCS_ZooDsBM2k3s_3akWtn1w,11,https://yt3.ggpht.com/ytc/APkrFKbOmNzktI61AIGK...,False



[1.04 seconds] Done!


PermissionError: [Errno 13] Permission denied: 'ENGLISH_youtube_comments.csv'

In [7]:
df_comment = pd.read_csv('ENGLISH_youtube_comments.csv')
df_comment.shape

(212, 9)